In [2]:
# !pip install findspark
import platform
print(platform.python_version())
# !pip install pyspark
import findspark
findspark.init()
import pyspark
findspark.find()
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySparkDemo").getOrCreate()


3.11.14


# importing module

In [3]:
from pyspark.sql import SparkSession

# creating sparksession and giving an app name

In [4]:
spark = SparkSession.builder.appName('sparkdf').getOrCreate()

# list  of students  data

In [5]:
data = [["1", "sravan", "vignan"],
        ["2", "ojaswi", "vvit"],
        ["3", "rohith", "vvit"],
        ["4", "sridevi", "vignan"],
        ["6", "ravi", "vrs"],
        ["5", "gnanesh", "iit"]]


# specify column names

In [6]:
columns = ['ID', 'NAME', 'college']

# creating a dataframe from the lists of data

In [7]:
dataframe = spark.createDataFrame(data, columns)
print('Actual data in dataframe')
dataframe.show()


Actual data in dataframe
+---+-------+-------+
| ID|   NAME|college|
+---+-------+-------+
|  1| sravan| vignan|
|  2| ojaswi|   vvit|
|  3| rohith|   vvit|
|  4|sridevi| vignan|
|  6|   ravi|    vrs|
|  5|gnanesh|    iit|
+---+-------+-------+



## Kwerendy SQL

In [8]:
# CREATE DATABASE IF NOT EXISTS grupa_db;
# USE grupa_db;

# CREATE TABLE IF NOT EXISTS koledzy (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     imie VARCHAR(100),
#     nazwisko VARCHAR(100),
#     email VARCHAR(100),
#     telefon VARCHAR(20),
#     plec VARCHAR(1),
#     wiek INT,
#     miasto VARCHAR(100),
#     ulubiona_technologia VARCHAR(100)
# );

# TRUNCATE TABLE koledzy;

# INSERT INTO koledzy 
#     (imie, nazwisko, email, telefon, plec, wiek, miasto, ulubiona_technologia) 
# VALUES
#     ('Daniel', 'Cachro', 'daniel.cachro@mail.com', '795 850 167', 'M', 24, 'Biały Dunajec', 'Python'),
#     ('Dariusz', 'Gał', 'dariusz.gal01@mail.com', '609 385 043', 'M', 23, 'Gliczarów Dolny', 'Java'),
#     ('Dawid', 'Morawa', 'dawid.morawa@mail.com', '602 131 322', 'M', 23, 'Nowy Targ', 'SQL'),
#     ('Dominik', 'Spytkowski', 'dominik.spytkowski@mail.com', '770 213 456', 'M', 24, 'Nowy-Targ', 'Spark'),
#     ('Jakub', 'Bąbol', 'jakub.babol07@mail.com', '539 514 346', 'M', 22, 'Rabka Zdrój', 'Python'),
#     ('Kacper', 'Siciarz', 'kacper.siciarz@mail.com', '764 234 121', 'M', 23, 'Nowy-Targ', 'SQL'),
#     ('Marta', 'Stuglik', 'marta.stuglik@mail.com', '531 292 782', 'K', 22, 'Czarny Dunajec', 'Spark'),
#     ('Maciej', 'Palenica', 'maciek.palenica011@mail.com', '511 153 069', 'M', 23, 'Dzianisz-City', 'Java'),
#     ('Michał', 'Kubacki', 'michal.kubacki@mail.com', '511 861 327', 'M', 22, 'Nowy-Targ', 'Python'),
#     ('Michał', 'Szewczyk', 'michal.szewczyk@mail.com', '234 316 643', 'M', 23, 'Nowy-Targ', 'Java'),
#     ('Piotr', 'Postrożny', 'piotr.postrozny@mail.com', '696 569 660', 'M', 23, 'Zakopane', 'SQL'),
#     ('Rafał', 'Łuszczek', 'rafal.luszczekk@mail.com', '531 297 764', 'M', 23, 'Zakopane', 'Spark'),
#     ('Wiesław', 'Horniczak', 'wieslaw.horniczak112@mail.com', '880 854 241', 'M', 23, 'Falsztyn', 'Python');

# SELECT * FROM koledzy;

## Operacje na Bazie Danych

In [9]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

SCIEZKA_DO_STEROWNIKA_JAR = "mysql-connector.jar" 

DB_PASS = "1234"

DB_USER = "root"
DB_URL = "jdbc:mysql://localhost:3306/grupa_db"
DB_TABLE = "koledzy"


print(">>> Próba uruchomienia sesji Spark ze sterownikiem MySQL...")

try:
    spark = SparkSession.builder \
        .appName("MySQL_Zadanie_Lab") \
        .config("spark.driver.extraClassPath", SCIEZKA_DO_STEROWNIKA_JAR) \
        .getOrCreate()
    
    print("--- SUKCES! Sesja Spark uruchomiona. --- \n")

except Exception as e:
    print(f"BŁĄD KRYTYCZNY: Nie udało się uruchomić Sparka.")
    print("Sprawdź, czy nazwa pliku w 'SCIEZKA_DO_STEROWNIKA_JAR' jest poprawna.")
    print(e)
    raise SystemExit("Zatrzymano z powodu błędu Spark.")

print(f">>> Wczytywanie danych z tabeli '{DB_TABLE}'...")
try:
    df = spark.read \
        .format("jdbc") \
        .option("url", DB_URL) \
        .option("dbtable", DB_TABLE) \
        .option("user", DB_USER) \
        .option("password", DB_PASS) \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .load()

    print(f"--- SUKCES! Pobrano dane z MySQL. Oto one ({df.count()} osób): ---")
    df.show()

except Exception as e:
    print(f"BŁĄD: Nie udało się odczytać danych z MySQL.")
    print("Sprawdź czy: 1. Serwer MySQL działa, 2. Hasło jest poprawne (DB_PASS), 3. Baza 'grupa_db' i tabela 'koledzy' istnieją.")
    print(e)
    spark.stop()
    raise SystemExit("Zatrzymano z powodu błędu bazy danych.")


# 4. WYFILTRUJ OSOBY, KTÓRYCH WIEK JEST WYŻSZY NIŻ 22 LATA
print("\n>>> Filtrowanie osób z wiekiem > 22:")

df_filtered = df.filter(df.wiek > 22)

print(f"--- SUKCES! Znaleziono {df_filtered.count()} osób starszych niż 22 lata: ---")
df_filtered.show()


# 5. DOPISZ DO BAZY DANYCH INFORMACJE O DODATKOWEJ OSOBIE
print("\n>>> Dodawanie nowej osoby ('Jan Testowy') do bazy MySQL...")

# Tworzymy dane dla nowej osoby
# Ważne: Lista kolumn musi pasować do bazy DOKŁADNIE, ale pomijamy 'id'
new_person_data = [
    ('Jan', 'Testowy', 'jan.test@mail.com', '999888777', 'M', 26, 'Zdalnie', 'PySpark')
]

# Tworzymy schemat dla nowej osoby (bez kolumny 'id')
new_person_schema = StructType([
    StructField("imie", StringType(), True),
    StructField("nazwisko", StringType(), True),
    StructField("email", StringType(), True),
    StructField("telefon", StringType(), True),
    StructField("plec", StringType(), True),
    StructField("wiek", IntegerType(), True),
    StructField("miasto", StringType(), True),
    StructField("ulubiona_technologia", StringType(), True)
])

new_person_df = spark.createDataFrame(data=new_person_data, schema=new_person_schema)

new_person_df.write \
    .format("jdbc") \
    .option("url", DB_URL) \
    .option("dbtable", DB_TABLE) \
    .option("user", DB_USER) \
    .option("password", DB_PASS) \
    .mode("append") \
    .save()

print("--- SUKCES! Nowa osoba dodana. ---")

print(f"\n>>> Weryfikacja. Pełna tabela '{DB_TABLE}' po aktualizacji:")
final_df = spark.read \
    .format("jdbc") \
    .option("url", DB_URL) \
    .option("dbtable", DB_TABLE) \
    .option("user", DB_USER) \
    .option("password", DB_PASS) \
    .load()

final_df.show()
print(f"Całkowita liczba osób w bazie: {final_df.count()}")
spark.stop()
print("\n>>> Sesja Spark zakończona. Zadanie wykonane.")

>>> Próba uruchomienia sesji Spark ze sterownikiem MySQL...
--- SUKCES! Sesja Spark uruchomiona. --- 

>>> Wczytywanie danych z tabeli 'koledzy'...
--- SUKCES! Pobrano dane z MySQL. Oto one (14 osób): ---
+---+-------+----------+--------------------+-----------+----+----+---------------+--------------------+
| id|   imie|  nazwisko|               email|    telefon|plec|wiek|         miasto|ulubiona_technologia|
+---+-------+----------+--------------------+-----------+----+----+---------------+--------------------+
|  1| Daniel|    Cachro|daniel.cachro@mai...|795 850 167|   M|  24|  Biały Dunajec|              Python|
|  2|Dariusz|       Gał|dariusz.gal01@mai...|609 385 043|   M|  23|Gliczarów Dolny|                Java|
|  3|  Dawid|    Morawa|dawid.morawa@mail...|602 131 322|   M|  23|      Nowy Targ|                 SQL|
|  4|Dominik|Spytkowski|dominik.spytkowsk...|770 213 456|   M|  24|      Nowy-Targ|               Spark|
|  5|  Jakub|     Bąbol|jakub.babol07@mai...|539 514 346|   